## Import

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import sys
sys.executable

'D:\\Anaconda\\python.exe'

In [3]:
import random as rnd
import warnings

import matplotlib
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Reading data:

datapath : "../data"

In [21]:
traindf = pd.read_csv('../data/train.csv', index_col="ID")
traindf.sample(2)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22361,Resort Hotel,1,5,2016,January,2,4,1,3,2,0.0,0,HB,PRT,Groups,Corporate,0,0,0,A,A,0,No Deposit,NaN,174.0,0,Transient-Party,64.303920,0,0,Canceled,2015-12-30
77732,City Hotel,0,1,2016,December,53,26,1,0,2,1.0,0,BB,PRT,Direct,Direct,0,0,0,D,D,0,No Deposit,14.0,NaN,0,Transient,136.173904,0,3,Check-Out,2016-12-27


### Drop columns not in test data and Clean missing data
* Drop columns: `is_canceled`, `reservation_status`, `reservation_status_date`, where they aren't in test data

In [22]:
testCol = set(pd.read_csv('../data/test.csv', index_col="ID").columns)
removeCols = [col for col in traindf.columns if col not in testCol]
removeCols.remove("adr")
print(removeCols)

['is_canceled', 'reservation_status', 'reservation_status_date']


In [23]:
traindf = traindf.drop(removeCols, axis=1)
traindf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
31925,City Hotel,3,2016,March,13,25,1,2,2,0.0,0,SC,ISR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,144.112289,0,1
89677,City Hotel,4,2017,March,12,19,2,1,1,0.0,0,BB,ISR,Groups,TA/TO,0,0,0,A,A,2,No Deposit,37.0,NaN,0,Transient-Party,29.006519,0,0


### Missing data
The below code shows column name of missing data

In [24]:
tmp = pd.to_numeric((traindf.isnull().sum() / traindf.shape[0]).map('{:,.2f}'.format))
tmp[tmp > 0]

country    0.01
agent      0.14
company    0.94
dtype: float64

* The company feature's 94% is missing. Because of that, this feature will be eliminated. 
* Another missing data has occurred in country and agent features. Since missing data of country is less than 1%, these data will replace with most frequent value. 
* However, the agent missing features are more than the country. For this feature, missing data will be imputed as 0

### Turn agent and company to categorical data

Since there        

In [25]:
# In this state, avoid too many feature, I just drop agent and country columns

traindf["children"] = traindf["children"].fillna(0)
traindf["country"] = traindf["country"].fillna(traindf["country"].mode().index[0])
traindf["agent"] = traindf["agent"].fillna("0")
traindf = traindf.drop(["company", "agent", "country"], axis=1)

In [26]:
traindf.sample(5)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,
42085,City Hotel,68,2016,May,21,21,2,3,2,0.0,0,BB,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,0,Transient,140.000242,0,0
13833,City Hotel,360,2015,October,42,12,1,1,2,0.0,0,BB,Groups,TA/TO,0,1,0,A,A,0,Non Refund,0,Contract,35.021344,0,0
30390,City Hotel,75,2016,March,12,17,1,3,1,0.0,0,BB,Offline TA/TO,TA/TO,0,0,0,A,A,0,Non Refund,0,Transient,69.556917,0,0
78309,Resort Hotel,61,2016,December,53,29,0,3,2,0.0,0,HB,Direct,Direct,1,0,2,D,D,1,No Deposit,0,Transient,155.477316,1,1
73445,City Hotel,82,2016,November,47,19,2,5,2,1.0,0,SC,Online TA,TA/TO,0,0,0,A,A,1,No Deposit,0,Transient,66.181869,0,3


### Feature Transformation

`children`: `children` + `babies`  
`hotel`: Convert hotel as `Resort Hotel` to 0, `City Hotel` to 1  

Create new feature `is_same_room`: Check if `reserved_room_type` is same as `assigned_room_type`(If so 1, else 0)

One hot encoding: `meal`  
One hot encoding `reserved_room_type` and `assigned_room_type`  
One hot encoding `arrival_date_month`  

In [27]:
traindf = pd.get_dummies(traindf, prefix="", prefix_sep="", columns=["arrival_date_month"])
traindf["children"] = traindf["children"] + traindf["babies"]
traindf = traindf.drop(["babies"], axis=1)
traindf["hotel"] = traindf["hotel"].map({"Resort Hotel": 0, "City Hotel": 1})

traindf["is_same_room"] = (traindf["reserved_room_type"] == traindf["assigned_room_type"]).map({True: 1, False: 0})
traindf = pd.get_dummies(traindf, columns=["reserved_room_type", "assigned_room_type"], prefix=["reserved", "assigned"])


traindf = pd.get_dummies(traindf, columns=["meal"])

In [28]:
traindf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,April,August,December,February,January,July,June,March,May,November,October,September,is_same_room,reserved_A,reserved_B,reserved_C,reserved_D,reserved_E,reserved_F,reserved_G,reserved_H,reserved_L,reserved_P,assigned_A,assigned_B,assigned_C,assigned_D,assigned_E,assigned_F,assigned_G,assigned_H,assigned_I,assigned_K,assigned_L,assigned_P,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
24949,0,2,2016,7,7,2,0,2,0.0,Direct,Direct,1,0,1,1,No Deposit,0,Transient,75.412935,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
42709,1,158,2016,22,24,0,2,1,0.0,Groups,TA/TO,0,0,0,0,Non Refund,31,Transient,111.555489,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


### Train Model and Evaluate best model performance:

Since I only know about regression, I only use SVR as my model, and save the result as 

In [33]:
traindf = pd.get_dummies(traindf)
new_train_x = traindf.drop("adr", axis=1).values
new_train_y = traindf["adr"].values

In [34]:
traindf = pd.get_dummies(traindf, columns=["arrival_date_year"])
traindf = traindf.drop(["arrival_date_week_number", "arrival_date_day_of_month"], axis=1)

In [36]:
traindf.to_csv("PredictAdr.csv")

In [29]:
# Scaler

#from sklearn.preprocessing import MinMaxScaler

#scaler = MinMaxScaler()
#print(scaler.fit(new_train_x))
#print(scaler.transform(new_train_x))
#print(scaler.transform(new_train_y))

In [30]:
X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y, test_size=0.3)
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [33]:
# The result here shows the avg E_in

np.linalg.norm((y_pred - y_test)) / len(y_pred)

0.17806214424870953

In [34]:
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(new_train_x, new_train_y)
y_pred = regr.predict(new_train_x)

In [35]:
y_pred_csv = pd.DataFrame(y_pred)

In [42]:
y_pred_csv.to_csv("adr_predict.csv", header=["adr"])

In [44]:
adr = pd.DataFrame(new_train_y)

In [45]:
y_pred_csv

,0
0,102.197540
1,60.598775
2,72.030726
3,72.030726
4,72.030726
...,...
91526,96.868011
91527,-2.138195
91528,84.689882
91529,44.351993


In [46]:
adr

,0
0,-6.305161
1,75.052227
2,74.546401
3,76.376288
4,49.411647
...,...
91526,79.223571
91527,-6.822102
91528,90.814554
91529,38.135565


In [47]:
diff = y_pred_csv - adr

In [50]:
diff = diff.apply(abs)

In [57]:
indexes = diff[diff[0] > 50].index

In [61]:
perform_bad = traindf.loc[indexes]

In [77]:
perform_bad[perform_bad["adr"] < 10]["adr"]

0        -6.305161
63       -0.729340
64       -2.902196
119     -88.642804
341      -7.897349
           ...    
91177     7.805044
91261   -62.157892
91263   -59.188184
91275   -68.646270
91305   -44.775831
Name: adr, Length: 1246, dtype: float64

In [78]:
traindf[traindf["adr"] < 10]["adr"]

ID
0        -6.305161
63       -0.729340
64       -2.902196
119     -88.642804
148       8.960203
           ...    
91283   -27.102503
91305   -44.775831
91500   -14.504251
91503   -12.919657
91527    -6.822102
Name: adr, Length: 4010, dtype: float64